# Long Term Credit Rating Projection 

## Import of all the packages needed

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV, cross_validate, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
import statsmodels.api as sm
from statsmodels.imputation.mice import MICE, MICEData
# import fancyimpute
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.impute import SimpleImputer
# from imblearn.over_sampling import SMOTE
import time
from collections import Counter
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
#import all the functions we wrote ourselves
import import_ipynb
import Functions as functions


ModuleNotFoundError: No module named 'import_ipynb'

## Importing and Preprocessing Data

Get the 3 provided csv and merge them into one Pandas-Dataframe.

In [2]:
#Add Keyfigures 
arr1=pd.read_csv(r'C:\Users\Sandro\OneDrive\Dokumente\Universität Zürich\Aufbaustufe\2020 FS\Introduction to Machine Learning\Group Project\Data\d02dfcb3d4b2bc42.csv')
#Add Ratings
arr2=pd.read_csv(r'C:\Users\Sandro\OneDrive\Dokumente\Universität Zürich\Aufbaustufe\2020 FS\Introduction to Machine Learning\Group Project\Data\Ratings.csv')
#Add S&P500 Company List
arr3=pd.read_csv(r'C:\Users\Sandro\OneDrive\Dokumente\Universität Zürich\Aufbaustufe\2020 FS\Introduction to Machine Learning\Group Project\Data\SP500_CompanyList.csv')
#Combine Keyfigures and S&P500 Company List
rest=functions.combine_keyfigures_and_Companies(arr1,arr3)
#Add Ratings to the mapping
mapping=functions.combine_Ratings_and_Rest(rest,arr2)

FileNotFoundError: [Errno 2] File b'C:\\Users\\Sandro\\OneDrive\\Dokumente\\Universit\xc3\xa4t Z\xc3\xbcrich\\Aufbaustufe\\2020 FS\\Introduction to Machine Learning\\Group Project\\Data\\d02dfcb3d4b2bc42.csv' does not exist: b'C:\\Users\\Sandro\\OneDrive\\Dokumente\\Universit\xc3\xa4t Z\xc3\xbcrich\\Aufbaustufe\\2020 FS\\Introduction to Machine Learning\\Group Project\\Data\\d02dfcb3d4b2bc42.csv'

Save the joint Dataframe for further use in a csv:

In [3]:
#mapping.to_csv(r'C:\Users\Sandro\OneDrive\Dokumente\Universität Zürich\Aufbaustufe\2020 FS\Introduction to Machine Learning\Group Project\Data\Clean Mapping.csv')

In [4]:
#df = pd.read_csv(r'C:\Users\Sandro\OneDrive\Dokumente\Universität Zürich\Aufbaustufe\2020 FS\Introduction to Machine Learning\Group Project\Data\Clean Mapping.csv', sep=';',
                #parse_dates=['adate', 'qdate', 'public_date', 'datadate'])
df = mapping
df=df.rename(columns={'public_date_x':'public_date'})
df=df.rename(columns={'TICKER_x':'TICKER'})
df=df.rename(columns={'TICKER_y':'tic'})
df= df.drop(['public_date_y'],1)

Deleting all the rows where splticrm has NA values.

In [5]:
df = df[df['splticrm'].notna()]

In [7]:
df = df[df['splticrm'] != 'D']
df = df[df['splticrm'] != 'CCC']
print(df.head())

   permno       adate     qdate public_date   CAPEI     bm     evm  \
0   10104  20090531.0  20091130  2010-01-31  26.386  0.252  10.110   
1   10104  20090531.0  20091130  2010-02-28  28.226  0.252  10.110   
2   10104  20090531.0  20091130  2010-03-31  29.464  0.252  10.110   
3   10104  20090531.0  20100228  2010-04-30  28.783  0.233  10.983   
4   10104  20090531.0  20100228  2010-05-31  25.096  0.233  10.983   

   pe_op_basic  pe_op_dil  pe_exi  ...     CUSIP  NWPERM    PRC  \
0       19.217     19.378  20.052  ...  68389X10       0  44.06   
1       20.542     20.714  21.435  ...  68389X10       0  44.06   
2       21.425     21.605  22.357  ...  68389X10       0  44.06   
3       21.378     21.556  23.096  ...  68389X10       0  44.06   
4       18.653     18.808  20.152  ...  68389X10       0  44.06   

              key    gvkey  splticrm    datadate         conm   tic      cusip  
0  2010-01-31ORCL  12142.0         A  20100131.0  ORACLE CORP  ORCL  68389X105  
1  2010-02-28O

In [6]:
Y = pd.factorize(df["splticrm"])[0]
print(np.bincount(Y))
print(pd.factorize(df["splticrm"])[1])
Y = pd.DataFrame(Y, columns=["Rating as Factor"])
print(Y)

[3656 1943  990  296 5455 3209  489 3882 4682  199   50  143  382 1069
 1648 1096  220    4    2]
Index(['A', 'A+', 'AA-', 'AAA', 'BBB', 'BBB-', 'AA', 'A-', 'BBB+', 'AA+',
       'CCC+', 'B-', 'B+', 'BB-', 'BB+', 'BB', 'B', 'D', 'CCC'],
      dtype='object')
       Rating as Factor
0                     0
1                     0
2                     0
3                     0
4                     0
...                 ...
29410                 8
29411                 8
29412                 8
29413                 8
29414                 8

[29415 rows x 1 columns]


After taking a first look at the data we see that for ratings D and CCC we only have 4, respectively 2 observations. Therefore we delete these values due to the very low number of observations.

### Handling missing data
We decided to drop columns which contain more than 10'000 (corresponds to roughly 30%)  NA values or zero values.

In [8]:
NAs = df.isnull().sum() > 10000
Zeros =  (df == 0).sum() > 10000
delNAs = df.columns[NAs] #drops PEG_trailing
delZeros= df.columns[Zeros] #drops rd_sale, adv_sale, staff_sale
df = df.drop(delNAs, axis=1)
df = df.drop(delZeros, axis=1)
df.to_csv(r'C:\Users\Sandro\OneDrive\Dokumente\Universität Zürich\Aufbaustufe\2020 FS\Introduction to Machine Learning\Group Project\Collection\ML_Finance-master\Clean Mapping.csv')

Create a column of Ratings which are lagging by 1 Month and merge it with the dataframe

In [9]:
nest_list=functions.create_lagging_Ratings_1M(r'C:\Users\Sandro\OneDrive\Dokumente\Universität Zürich\Aufbaustufe\2020 FS\Introduction to Machine Learning\Group Project\Collection\ML_Finance-master\Clean Mapping.csv')
dframe=pd.DataFrame(nest_list,columns=['key','Lagging-Rating_1M'])
#Replace the Nonetype from python with NaN from pandas
dframe.fillna(value=pd.np.nan, inplace=True)
#Now merge the new Dataframe to our main-df
df = df.merge(dframe, how='left', on=['key'])
df.head()

c:\users\sandro\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  after removing the cwd from sys.path.


,permno,adate,qdate,public_date,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,...,CUSIP,PRC,key,gvkey,splticrm,datadate,conm,tic,cusip,Lagging-Rating_1M
0,10104,20090531.0,20091130,2010-01-31,26.386,0.252,10.110,19.217,19.378,20.052,...,68389X10,44.06,2010-01-31ORCL,12142.0,A,20100131.0,ORACLE CORP,ORCL,68389X105,NaN
1,10104,20090531.0,20091130,2010-02-28,28.226,0.252,10.110,20.542,20.714,21.435,...,68389X10,44.06,2010-02-28ORCL,12142.0,A,20100228.0,ORACLE CORP,ORCL,68389X105,A
2,10104,20090531.0,20091130,2010-03-31,29.464,0.252,10.110,21.425,21.605,22.357,...,68389X10,44.06,2010-03-31ORCL,12142.0,A,20100331.0,ORACLE CORP,ORCL,68389X105,A
3,10104,20090531.0,20100228,2010-04-30,28.783,0.233,10.983,21.378,21.556,23.096,...,68389X10,44.06,2010-04-30ORCL,12142.0,A,20100430.0,ORACLE CORP,ORCL,68389X105,A
4,10104,20090531.0,20100228,2010-05-31,25.096,0.233,10.983,18.653,18.808,20.152,...,68389X10,44.06,2010-05-31ORCL,12142.0,A,20100531.0,ORACLE CORP,ORCL,68389X105,A


Assign to X all the columns but splticrm. Then we drop some columns which will not be relevant for the algorithms.

In [10]:
X = df.loc[:, df.columns != "splticrm"]
X = X.drop(["permno", "CUSIP", "NCUSIP", "adate", "qdate", "public_date", "TICKER"], axis=1)
X = X.drop(["COMNAM", "PERMCO", "gvkey", "datadate", "tic", "cusip", "conm", "PRC"], axis=1)
X.head()

,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps,pcf,dpr,...,sale_equity,sale_nwc,accrual,ptb,DIVYIELD,PEG_1yrforward,PEG_ltgforward,date,key,Lagging-Rating_1M
0,26.386,0.252,10.110,19.217,19.378,20.052,20.052,4.976,13.353,0.129,...,0.912,1.894,0.058,4.145,.867%,2.221,1.554,29/06/2018,2010-01-31ORCL,NaN
1,28.226,0.252,10.110,20.542,20.714,21.435,21.435,5.323,14.285,0.129,...,0.912,1.894,0.058,4.434,.811%,2.058,1.520,29/06/2018,2010-02-28ORCL,A
2,29.464,0.252,10.110,21.425,21.605,22.357,22.357,5.556,14.911,0.129,...,0.912,1.894,0.058,4.628,.778%,2.146,1.586,29/06/2018,2010-03-31ORCL,A
3,28.783,0.233,10.983,21.378,21.556,23.096,23.096,5.381,15.909,0.177,...,0.902,1.891,0.049,4.515,.773%,1.848,1.642,29/06/2018,2010-04-30ORCL,A
4,25.096,0.233,10.983,18.653,18.808,20.152,20.152,4.692,13.871,0.177,...,0.902,1.891,0.049,3.937,.886%,1.612,1.432,29/06/2018,2010-05-31ORCL,A


### Handling missing data
We decided to drop columns which contain more than 10'000 (corresponds to roughly 30%)  NA values or zero values.

In [11]:
'''
NAs = X.isnull().sum() > 10000
Zeros =  (X == 0).sum() > 10000
delNAs = X.columns[NAs] #drops PEG_trailing
delZeros= X.columns[Zeros] #drops rd_sale, adv_sale, staff_sale
X = X.drop(delNAs, axis=1)
X = X.drop(delZeros, axis=1)
'''

'\nNAs = X.isnull().sum() > 10000\nZeros =  (X == 0).sum() > 10000\ndelNAs = X.columns[NAs] #drops PEG_trailing\ndelZeros= X.columns[Zeros] #drops rd_sale, adv_sale, staff_sale\nX = X.drop(delNAs, axis=1)\nX = X.drop(delZeros, axis=1)\n'

Now we split the data into a train and a test set. The test set consists of 20% of the whole dataset.

In [12]:
#now do the train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, 
                                                    test_size=0.2, 
                                                    random_state=0, 
                                                    stratify=Y)

ValueError: Found input variables with inconsistent numbers of samples: [29409, 29415]

In [ ]:
X_train.head(1)


In [ ]:
X_train.isna().sum()

We see that there are still missing values that have to be imputed. We do this in the next section.

#### Iterative Imputer
The iterative imputer imputes missing values by by modeling each feature containing missing values as a function of other features and is applied separately to the train and test set.

In [ ]:
# apply IterativeImputer
## this can be deleted once the functions file is imported

num_cols = ['CAPEI', 'bm', 'evm', 'pe_op_basic', 'pe_op_dil', 'pe_exi', 'pe_inc', 'ps', 'pcf', 
            'dpr', 'npm', 'opmbd', 'opmad', 'gpm', 'ptpm', 'cfm', 'roa', 'roe', 'roce', 'efftax', 'aftret_eq',
            'aftret_invcapx', 'aftret_equity', 'pretret_noa', 'pretret_earnat', 'GProf', 'equity_invcap',
            'debt_invcap', 'totdebt_invcap', 'capital_ratio', 'int_debt', 'int_totdebt', 'cash_lt', 'invt_act',
            'rect_act', 'debt_at', 'debt_ebitda', 'short_debt', 'curr_debt', 'lt_debt', 'profit_lct', 'ocf_lct',
            'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be', 'debt_assets', 'debt_capital', 'de_ratio', 'intcov',
            'intcov_ratio', 'cash_ratio', 'quick_ratio', 'curr_ratio', 'cash_conversion', 'inv_turn', 'at_turn',
            'rect_turn', 'pay_turn', 'sale_invcap', 'sale_equity', 'sale_nwc', 'accrual', 'ptb',
            'DIVYIELD', 'PEG_1yrforward', 'PEG_ltgforward']

# Copy df to df_mice_imputed
X_train_imputed = X_train[num_cols].copy(deep=True)

# Initialize IterativeImputer
mice_imputer = IterativeImputer(random_state=0)

# Impute using fit_tranform on diabetes
X_train_imputed.iloc[:, :] = mice_imputer.fit_transform(X_train[num_cols])

# Copy df to df_mice_imputed
X_test_imputed = X_test[num_cols].copy(deep=True)

# Impute using tranformation of training set on test set
X_test_imputed.iloc[:, :] = mice_imputer.transform(X_test[num_cols])

In [ ]:
# At the latest here import the 'Functions' file

In [ ]:
# apply IterativeImputer

X_train_imputed = my_iterative_imputer(X_train)
X_test_imputed = my_iterative_imputer(X_test)

#### Feature Selection
To find out which variables are most important we run the 'features_selection' function and select all variables which explain more than 0.15%.

In [ ]:
important_features = feature_selection(x = X_train_imputed, y = y_train.values.ravel(), thres=0.015)

In [ ]:
#Subset important features matrix for ML algorithms
X_train_imputed = X_train_imputed.loc[:,important_features]
X_test_imputed = X_test_imputed.loc[:, important_features]

In [ ]:
X_train_imputed.head(5)

In [ ]:
X_train_imputed.to_csv('X_train.csv')
X_test_imputed.to_csv('X_test.csv')
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')

## Machine Learning Algorithms

### Logistic Regression

In [ ]:
## Logistic Regression

X_train = pd.read_csv("X_train.csv")
X_train = X_train.iloc[:, 1:]
X_test = pd.read_csv("X_test.csv")
X_test = X_test.iloc[:, 1:]
Y_train = pd.read_csv("y_train.csv")
Y_test = pd.read_csv("y_test.csv")
Y_train = Y_train['Rating as Factor'].astype('category') #factorize trainset
Y_test = Y_test['Rating as Factor'].astype('category')   #factorize testset

In [ ]:
import datetime
print(datetime.datetime.now())
grid = LogReg(X_train,Y_train)
print('Best parameters:', grid.best_params_) #best parameters are C=7 & ratio=0 -> l2 penalty function
print('Best CV accuracy:', grid.best_score_)
print('Test score:', grid.score(X_test,Y_test)) #31%
print(datetime.datetime.now()) #10 minutes

In [ ]:
# Predict classes
y_pred = grid.predict(X_test)

# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test})
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted'))

### Support Vector Machines

In [ ]:
#######Polynomial Kernel Function#######
print(datetime.datetime.now()) #computation time
poly = SVM_poly(X_train,Y_train)
print('Best CV accuracy: {:.2f}'.format(poly.best_score_))
print('Test score:       {:.2f}'.format(poly.score(X_test, Y_test)))
print('Best parameters: {}'.format(poly.best_params_))
print(datetime.datetime.now()) #20min

# Predict classes
y_pred = poly.predict(X_test)

# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test})
print('Polynomial Kernel Function yields the following confusion matrix:')
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted')) 


#######Radial Basis Kernel Function(rbf)#######
print(datetime.datetime.now()) #computation time
rbf = SVM_rbf(X_train,Y_train)
print('Best CV accuracy: {:.2f}'.format(rbf.best_score_))
print('Test score:       {:.2f}'.format(rbf.score(X_test, Y_test)))
print('Best parameters: {}'.format(rbf.best_params_))
print(datetime.datetime.now()) #10min

# Predict classes
y_pred = rbf.predict(X_test)

# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test})
print('Radial Basis Function Kernel yields the following confusion matrix:')
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted'))


#######Radial Basis Kernel Function(rbf) with Balanced class weights#######
print(datetime.datetime.now()) #computation time
rbf_bal = SVM_rbf_bal(X_train,Y_train)
print('Best CV accuracy: {:.2f}'.format(rbf_bal.best_score_))
print('Test score:       {:.2f}'.format(rbf_bal.score(X_test, Y_test)))
print('Best parameters: {}'.format(rbf_bal.best_params_))
print(datetime.datetime.now())#10min

#looking at the confusion matrix of non-balanced rbf we see that the smaller classes don't get more wrong classification. Therefore balancing the weights should not influence the outcome greatly which it doesn't


# Predict classes
y_pred = rbf_bal.predict(X_test)

# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': y_pred,
                      'True': Y_test})
print('Radial Basis Function Kernel with Balanced class weights yields the following confusion matrix:')
print(confm.groupby(['True','Predicted'], sort=True).size().unstack('Predicted'))

#According to "https://stackoverflow.com/questions/21390570/scikit-learn-svc-coef0-parameter-range" the Sigmoid function does not fulfill the definition of a kernel as it is not positive semidefinite. Therefore we will not use it with Support Vector Machines.